In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('car-details.csv')
df.sample(5)

,name,company,model,edition,year,owner,fuel,seller_type,transmission,km_driven,mileage_mpg,engine_cc,max_power_bhp,torque_nm,seats,selling_price
4643,Volkswagen Vento Diesel Highline,Volkswagen,Vento,Diesel Highline,2011,Second,Diesel,Individual,Manual,100000,48.28,1598.0,103.60,250.0,5.0,370000
3269,Hyundai Santro Xing XL,Hyundai,Santro,Xing XL,2007,First,Petrol,Individual,Manual,64000,NaN,NaN,NaN,NaN,NaN,90000
3631,Renault Duster 110PS Diesel RxL,Renault,Duster,110PS Diesel RxL,2012,Second,Diesel,Individual,Manual,110000,44.70,1461.0,108.45,248.0,5.0,500000
4366,Maruti Wagon R AMT VXI,Maruti,Wagon,R AMT VXI,2018,Second,Petrol,Individual,Automatic,5000,48.20,998.0,67.00,90.0,5.0,445000
3590,Mahindra XUV300 W8 Option BSIV,Mahindra,XUV300,W8 Option BSIV,2019,First,Petrol,Individual,Manual,22000,39.94,1197.0,110.00,200.0,5.0,1000000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6926 entries, 0 to 6925
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           6926 non-null   object 
 1   company        6926 non-null   object 
 2   model          6926 non-null   object 
 3   edition        6926 non-null   object 
 4   year           6926 non-null   int64  
 5   owner          6926 non-null   object 
 6   fuel           6926 non-null   object 
 7   seller_type    6926 non-null   object 
 8   transmission   6926 non-null   object 
 9   km_driven      6926 non-null   int64  
 10  mileage_mpg    6718 non-null   float64
 11  engine_cc      6718 non-null   float64
 12  max_power_bhp  6717 non-null   float64
 13  torque_nm      6717 non-null   float64
 14  seats          6718 non-null   float64
 15  selling_price  6926 non-null   int64  
dtypes: float64(5), int64(3), object(8)
memory usage: 865.9+ KB


In [4]:
df.isna().sum()

name               0
company            0
model              0
edition            0
year               0
owner              0
fuel               0
seller_type        0
transmission       0
km_driven          0
mileage_mpg      208
engine_cc        208
max_power_bhp    209
torque_nm        209
seats            208
selling_price      0
dtype: int64

In [5]:
df.shape

(6926, 16)

In [6]:
for col in df.select_dtypes(include='object').columns:
    print(f"col:{col}")
    print(f"Cardinality:{df[col].nunique()}")
    print(df[col].unique())
    print(df[col].value_counts(normalize=True))
    print()

col:name
Cardinality:2058
['Maruti Swift Dzire VDI' 'Skoda Rapid 1.5 TDI Ambition'
 'Honda City 2017-2020 EXi' ... 'Tata Nexon 1.5 Revotorq XT'
 'Ford Freestyle Titanium Plus Diesel BSIV'
 'Toyota Innova 2.5 GX (Diesel) 8 Seater BS IV']
name
Maruti Swift Dzire VDI                       0.017037
Maruti Alto 800 LXI                          0.010973
Maruti Alto LXi                              0.009962
Maruti Swift VDI                             0.008663
Maruti Swift VDI BSIV                        0.008085
                                               ...   
Tata Tiago 1.2 Revotron XZ Plus Dual Tone    0.000144
Mahindra KUV 100 mFALCON G80 K4 Plus 5str    0.000144
Hyundai Verna SX Diesel                      0.000144
Maruti S-Presso VXI AT                       0.000144
Hyundai Santro Xing XK eRLX EuroIII          0.000144
Name: proportion, Length: 2058, dtype: float64

col:company
Cardinality:32
['Maruti' 'Skoda' 'Honda' 'Hyundai' 'Toyota' 'Ford' 'Renault' 'Mahindra'
 'Tata' 'Chevrol

In [7]:
df=df.drop(columns=['name','model','edition'])
df

,company,year,owner,fuel,seller_type,transmission,km_driven,mileage_mpg,engine_cc,max_power_bhp,torque_nm,seats,selling_price
0,Maruti,2014,First,Diesel,Individual,Manual,145500,55.00,1248.0,74.00,190.000000,5.0,450000
1,Skoda,2014,Second,Diesel,Individual,Manual,120000,49.70,1498.0,103.52,250.000000,5.0,370000
2,Honda,2006,Third,Petrol,Individual,Manual,140000,41.60,1497.0,78.00,124.544455,5.0,158000
3,Hyundai,2010,First,Diesel,Individual,Manual,127000,54.06,1396.0,90.00,219.668960,5.0,225000
4,Maruti,2007,First,Petrol,Individual,Manual,120000,37.84,1298.0,88.20,112.776475,5.0,130000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6921,Maruti,2013,Second,Petrol,Individual,Manual,50000,44.40,998.0,67.10,90.000000,5.0,260000
6922,Hyundai,2014,Second,Diesel,Individual,Manual,80000,52.97,1396.0,88.73,219.700000,5.0,475000
6923,Hyundai,2013,First,Petrol,Individual,Manual,110000,43.47,1197.0,82.85,113.700000,5.0,320000
6924,Hyundai,2007,Fourth & Above,Diesel,Individual,Manual,119000,39.47,1493.0,110.00,235.359600,5.0,135000


In [8]:
df.duplicated().sum()

np.int64(19)

In [9]:
df=df.drop_duplicates()
df

,company,year,owner,fuel,seller_type,transmission,km_driven,mileage_mpg,engine_cc,max_power_bhp,torque_nm,seats,selling_price
0,Maruti,2014,First,Diesel,Individual,Manual,145500,55.00,1248.0,74.00,190.000000,5.0,450000
1,Skoda,2014,Second,Diesel,Individual,Manual,120000,49.70,1498.0,103.52,250.000000,5.0,370000
2,Honda,2006,Third,Petrol,Individual,Manual,140000,41.60,1497.0,78.00,124.544455,5.0,158000
3,Hyundai,2010,First,Diesel,Individual,Manual,127000,54.06,1396.0,90.00,219.668960,5.0,225000
4,Maruti,2007,First,Petrol,Individual,Manual,120000,37.84,1298.0,88.20,112.776475,5.0,130000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6921,Maruti,2013,Second,Petrol,Individual,Manual,50000,44.40,998.0,67.10,90.000000,5.0,260000
6922,Hyundai,2014,Second,Diesel,Individual,Manual,80000,52.97,1396.0,88.73,219.700000,5.0,475000
6923,Hyundai,2013,First,Petrol,Individual,Manual,110000,43.47,1197.0,82.85,113.700000,5.0,320000
6924,Hyundai,2007,Fourth & Above,Diesel,Individual,Manual,119000,39.47,1493.0,110.00,235.359600,5.0,135000


In [10]:
df.duplicated().sum()

np.int64(0)

In [11]:
x=df.drop(columns='selling_price')
y=df.selling_price.copy()
print(x.shape,y.shape)

(6907, 12) (6907,)


In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(x_train.shape,x_test.shape)
print(y_train.shape,y_test.shape)

(5525, 12) (1382, 12)
(5525,) (1382,)


In [13]:
num_cols=x_train.select_dtypes(include='number').columns.tolist()
cat_cols=[col for col in x_train.columns if col not in num_cols]
print(num_cols)
print(cat_cols)

['year', 'km_driven', 'mileage_mpg', 'engine_cc', 'max_power_bhp', 'torque_nm', 'seats']
['company', 'owner', 'fuel', 'seller_type', 'transmission']


In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor

# Numerical pipeline
num_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Categorical pipeline
cat_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Column transformer
preprocessor = ColumnTransformer(transformers=[
    ('num', num_pipe, num_cols),
    ('cat', cat_pipe, cat_cols)
])

# Regressor
regressor = RandomForestRegressor(
    n_estimators=10, max_depth=5, random_state=42
)

# Full pipeline
rf_model = Pipeline(steps=[
    ('pre', preprocessor),
    ('reg', regressor)
])

# Fit the model
rf_model.fit(x_train, y_train)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'km_driven',
                                                   'mileage_mpg', 'engine_cc',
                                                   'max_power_bhp', 'torque_nm',
                                                   'seats']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['company', 'owner', 'fuel',
                                                   'seller_type',
                                                   'transmission'])])),
                ('reg',
                 RandomForestRegressor(max_depth=5, n_estimators=10,
                                       random_state=42))])

In [15]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Train RMSE
y_train_pred = rf_model.predict(x_train)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(f"train rmse : {train_rmse:,.3f}")
# Test RMSE
y_test_pred = rf_model.predict(x_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
print(f"test rmse : {test_rmse:,.3f}")

print("Train RMSE:", train_rmse)
print("Test RMSE:", test_rmse)

train rmse : 169,947.490
test rmse : 172,392.131
Train RMSE: 169947.48964050272
Test RMSE: 172392.1313605195
